In [ ]:
from pathlib import Path

from matplotlib.pyplot import rc

from skrt import Dose, Patient
from skrt.core import get_data_by_filename, relative_path
from voxtox.roi_names import head_and_neck_plan

In [ ]:
# Set Matplotlib runtime configuration.
# For details of Matplotlib configuration, see:
# https://matplotlib.org/stable/tutorials/introductory/customizing.html
# Sizes are in points.

# For axes, set spacing (pad) and size of label and title
rc("axes", labelpad=0, labelsize=25, titlepad=17, titlesize=25);

# Set default text charactieristics.
# Possible weight values are:
# 100, 200, 300, 400 / "normal", 500, 600, 700 / "bold", 800, 900.
rc("font", family="serif", serif=["Times"], size=20, weight=400);

# Set font size for legends.
rc("legend", fontsize=20)

# Set mathematics typeface when using matplotlib's built-in fonts.
rc("mathtext", fontset="dejavuserif");

# Use TeX/LaTeX for typesetting.  (This requires working TeX/LaTeX installation.)
rc("text", usetex=True)

# For ticks, set label size and direction ("in", "out", "inout").
rc(("xtick", "ytick"), labelsize=25, direction="out");

# For major and minor ticks, set size and width.
# For major ticks, set spacing (pad) of label.
rc(("xtick.major"), pad=3);
rc(("xtick.major", "ytick.major"), size=9, width=1.0);
rc(("xtick.minor", "ytick.minor"), size=4.5, width=1.0);
rc(("ytick.major"), pad=2);

# Create dictionary of BetterViewer image-display options.
view_opts = {
    # Set figure size in inches.
    "figsize": (18, 8),
    # Show major ticks at specified interval (axis units).
    "major_ticks": 60,
    # Show minor ticks for specified number of intervals between major ticks.
    "minor_ticks": 5,
    # Indicate whether axis units should be mm or numbers of voxels.
    "scale_in_mm" : True,
    # Indicate whether ticks should be shown on all sides.
    "ticks_all_sides": True,
    # Include y-tick labels for all plots, not only the first.
    "ytick_labels_first_only": False,
    # Specify zoom factor.
    "zoom": 1.5,
    # Set grey-level minimum and maximum (Hounsfield units).
    "intensity": (-200, 300),
    # Show probability scale.
    "colorbar": -2,
    # Overlay default image annotation (slice z-coordinate), in white, at default position (top left or image).
    "annotate_slice": {"color": "white", "fontsize": 28},
    # Make display interactive ("no_ui": False) or non-interactive ("no_ui": True).
    "no_ui": False,
    # Set title.
    "title": "",
};

In [ ]:
# Define data locations.
dcm_dir = Path('~/data/InnerEye/demo/project_data_2022_test_dicom').expanduser()
nii_dir = Path('~/data/InnerEye/demo/innereye_results/head_and_neck_pg_sc_dice').expanduser()
patient_dirs = sorted([patient_dir for patient_dir in nii_dir.glob('V*') if (dcm_dir / patient_dir.name).exists()])
print(f"Number of patient directories: {len(patient_dirs)}")

In [ ]:
# Set index of patient directory to be considered.
idx = 0
# Create patient object, and extract structure set, from NIfTI data (InnerEye output).
p1 = Patient(patient_dirs[idx])
ss1 = p1.studies[0].cthd_structure_sets[0]
# Create patient object, and extract structure set, from DICOM data.
p2 = Patient(dcm_dir / p1.id)
ss2 = p2.studies[0].ct_structure_sets[0].filtered_copy(names=head_and_neck_plan, keep_renamed_only=True)
ss2.name = "plan"
# Create combined structure set.
ss3 = ss1 + ss2

In [ ]:
# Not part of the workflow - just to show the different source types...
print(relative_path(ss1.get_rois()[0].path))
print(relative_path(ss2.path))

In [ ]:
# Compare ROIs.
ss1.get_comparison(ss2, metrics=['dice', 'jaccard', 'centroid', 'volume_ratio',
                                 'mean_surface_distance', 'hausdorff_distance'])

In [ ]:
# Obtain dictionary of InnerEye posterior probabilities and uncertainty (Shannon entropy).
# Cast to doses to allow overlay.
innereye_images = get_data_by_filename(p1.studies[0].innereye_images)
for key in innereye_images:
    image = innereye_images[key]
    innereye_images[key] = Dose(image.get_data(standardise=True) / 255,
                                affine=image.get_affine(standardise=True))
    if "posterior" in key:
        innereye_images[key]._default_colorbar_label = "Probability"
    elif "uncertainty" in key:
        innereye_images[key]._default_colorbar_label = "Shannon entropy"

print(innereye_images.keys())

In [ ]:
# Overlay posterior probability for spinal cord on CT image.
ss2.image.view(dose=innereye_images['001_posterior_spinal_cord'],
               rois=ss3, dose_kwargs={"vmin": 0, "vmax": 1},
               legend=True, legend_loc="center left", legend_bbox_to_anchor=(1.5, 0.5),
               **view_opts);